### This notebook includes the experiments for cross validation.

- Standing agent is P1, we are learning P2 (P2 is human player).

- Some frames will be removed, there are frames that;
    - Both players standing.
    - Consecutive ones(If frame's action is same with previous frame, it will be removed).
    - If we are(P2) in RECOV frame(Since we don't make RECOV frames ourselves).

### Importing libraries

In [20]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import TensorBoard
import time
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import json

# For using core package that located in the two upper folder.
import os,sys
sys.path.append('../../')

from core.json_importer import parse_json_file, parse_all_files
from core.filters import remove_both_standing_frames, remove_same_consecutive_actions, remove_recov_frames
from core.actions import one_hot_encode, decode
from core.preproccessing import Normalizer
from core.helpers import write_file

In [21]:
# Create out file folder if not exists
OUT_FOLDER = 'generated_files'
if not os.path.exists(OUT_FOLDER):
    os.makedirs(OUT_FOLDER)

In [22]:
# Open files
training_data = parse_all_files("data/train")
testing_data = parse_all_files("data/test")

### Pre process data

- In the pre process phase, we remove P2's datas from training set in order to crate labels. Labels are just actions of P2.

- We make actions one-hot encoding. 

- One-hot encoding for inputs and labels are not same. For example, Dash action in the input may have encoding [0,1] while Dash action in the label have encoding [0, 0, 1, 0].

- Integer values normalized. (P1-HP, P2-HP, P1-X, P1-Y, P2-X, P2-Y)

- After pre-process we just have <font color='red'>[P1-Action, P1-HP, P2-HP, P1-X, P1-Y, P2-X, P2-Y] -> P2-Action</font>

#### Remove unneeded frames

In [4]:
# Apply filters for training data
tr_deleted = training_data.filter(remove_both_standing_frames)
tr_deleted += training_data.filter(remove_recov_frames)
tr_deleted += training_data.filter(remove_same_consecutive_actions)

# Apply filters for training data
te_deleted = testing_data.filter(remove_both_standing_frames)
te_deleted += testing_data.filter(remove_recov_frames)
te_deleted += testing_data.filter(remove_same_consecutive_actions)

print('There are {} train data.({} deleted.)'.format(len(training_data), tr_deleted))
print('There are {} test data.({} deleted.)'.format(len(testing_data), te_deleted))

There are 11428 train data.(485372 deleted.)
There are 1221 test data.(52779 deleted.)


#### Encoding and normalization

In [6]:
def normalize_and_save(data, file_name):
    p1_hp_normalizer = Normalizer()
    p1_normalized_hp = p1_hp_normalizer.normalize(data)
    p1_hp_normalizer.save(OUT_FOLDER + '/' + file_name)
    return p1_normalized_hp

def process_data(game_data_obj):
    ## Pre process data
    processed_data = []

    # Create one hot encoding for actions (For input and labels)
    p1_one_hot_encoded_actions = one_hot_encode(game_data_obj.get_column("P1-action"))
    labels = one_hot_encode(game_data_obj.get_column("P2-action"))

    # Normalize uncategorized features
    p1_normalized_hp = normalize_and_save(game_data_obj.get_column("P1-hp"), "p1_hp_norm.save")
    p2_normalized_hp = normalize_and_save(game_data_obj.get_column("P2-hp"), "p2_hp_norm.save")
    normalized_xp1_distance = normalize_and_save([frame["P1-x"] for frame in game_data_obj], "xp1_norm.save")
    normalized_xp2_distance = normalize_and_save([frame["P2-x"] for frame in game_data_obj], "xp2_norm.save")
    normalized_yp1_distance = normalize_and_save([frame["P1-y"] for frame in game_data_obj], "yp1_norm.save")
    normalized_yp2_distance = normalize_and_save([frame["P2-y"] for frame in game_data_obj], "yp2_norm.save")

    for index in range(len(game_data_obj)):    
        processed_row = []
        processed_row.extend(p1_one_hot_encoded_actions[index])
        processed_row.extend(p1_normalized_hp[index]) 
        processed_row.extend(p2_normalized_hp[index])
        processed_row.extend(normalized_xp1_distance[index])
        processed_row.extend(normalized_xp2_distance[index])
        processed_row.extend(normalized_yp1_distance[index])
        processed_row.extend(normalized_yp2_distance[index])
        processed_data.append(processed_row)
    processed_data = np.array(processed_data)
    labels = np.array(labels)
    
    return processed_data, labels
    
tr_data, tr_labels = process_data(training_data)
te_data, te_labels = process_data(testing_data)

### Example data

In [7]:
EXAMPLE_ROW = 10

print("There are %d frames in dataset." % len(tr_data))
print("After pre processing the shape of our dataset is %s" % str(tr_data.shape))
print("\nOne example in index %d." % EXAMPLE_ROW)

row = tr_data[EXAMPLE_ROW]
print("\tProcessed Frame:" )
print("\t\tP1 Action(one-hot) Shape:\t[1, %d]" % 54)
print("\t\tP1 Hp(norm.):\t\t\t%s" % row[55])
print("\t\tP2 Hp(norm.):\t\t\t%s" % row[56])
print("\t\tX Dist(norm.):\t\t\t%s" % row[57])
print("\t\tY Dist(norm.):\t\t\t%s" % row[58])
print("\t\tLabel Shape:\t\t\t[1, %d]" % len(tr_labels[EXAMPLE_ROW]))

There are 11428 frames in dataset.
After pre processing the shape of our dataset is (11428, 61)

One example in index 10.
	Processed Frame:
		P1 Action(one-hot) Shape:	[1, 54]
		P1 Hp(norm.):			0.01594896331738437
		P2 Hp(norm.):			0.010683760683760684
		X Dist(norm.):			0.0425
		Y Dist(norm.):			0.08625000000000001
		Label Shape:			[1, 55]


### Neural Network Design

Our neural network has two hidden layers in this test. They has 12 and 8 neurons respectively.

In [8]:
# Constants
INPUT_LAYER_SIZE = tr_data.shape[1]
OUTPUT_LAYER_SIZE = tr_labels.shape[1]

In [9]:
def create_model(neuron_count):
    model = Sequential()
    model.add(Dense(neuron_count, input_dim=INPUT_LAYER_SIZE, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(OUTPUT_LAYER_SIZE, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [18]:
COMPLEXITIES = [1500, 750, 350, 175, 75, 25]
EPOCH = 500
BATCH = 10

In [19]:
if not os.path.exists(OUT_FOLDER):
    os.makedirs(OUT_FOLDER)

# Create session folder.
SESSION_FOLDER = OUT_FOLDER + '/session_{}'.format(time.time())

os.makedirs(SESSION_FOLDER)
os.makedirs(SESSION_FOLDER + '/models')
os.makedirs(SESSION_FOLDER + '/logs')
os.makedirs(SESSION_FOLDER + '/histories')

histories = []
# Create and run networks
for complexity in COMPLEXITIES:
    starting_time = time.clock()
    file_name = str(complexity)
    
    model = create_model(complexity)
    print('Model created with {} neurons.'.format(complexity))
    
    # Logging for tensorboard
    tensorboard = TensorBoard(log_dir= SESSION_FOLDER + "/logs/log_{}".format(file_name))
    
    print('\tTraining started.')
    history = model.fit(tr_data, tr_labels, epochs=EPOCH, validation_split=0.2,
                    shuffle=True, batch_size=BATCH, callbacks=[tensorboard])
    print('\tTraining ended.')
    
    # Display result
    scores = model.evaluate(te_data, te_labels)
    print("\t\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

    time_taken = time.clock() - starting_time * 1000.0
    
    model_info = {
        'info': 'One hidden layer ANN with {} neurons'.format(complexity),
        'epoch': EPOCH,
        'batch': BATCH,
        'time_taken': time_taken
    }
    
    # Save model and history
    model.save(SESSION_FOLDER + '/models/model_{}.h5'.format(file_name))
    write_file(SESSION_FOLDER + '/models/config_{}.json'.format(file_name), model.to_json())
    write_file(SESSION_FOLDER + '/models/info_{}.json'.format(file_name), json.dumps(model_info))
    
    pickle.dump(history, open(SESSION_FOLDER + '/histories/history_{}.save'.format(file_name), 'wb'))
    
    histories.append(history)
    
    
print('Session completed successfully. {} networks created and their data saved.'.format(len(COMPLEXITIES)))

Model created with 1500 neurons.
	Training started.
Train on 9142 samples, validate on 2286 samples
Epoch 1/500
9142/9142 [==============================] - 8s 908us/step - loss: 0.0701 - acc: 0.9815 - val_loss: 0.0557 - val_acc: 0.9827
Epoch 2/500
9142/9142 [==============================] - 7s 719us/step - loss: 0.0554 - acc: 0.9826 - val_loss: 0.0542 - val_acc: 0.9827
Epoch 3/500
9142/9142 [==============================] - 5s 544us/step - loss: 0.0538 - acc: 0.9829 - val_loss: 0.0528 - val_acc: 0.9829
Epoch 4/500
9142/9142 [==============================] - 5s 508us/step - loss: 0.0528 - acc: 0.9829 - val_loss: 0.0520 - val_acc: 0.9831
Epoch 5/500
9142/9142 [==============================] - 5s 507us/step - loss: 0.0520 - acc: 0.9831 - val_loss: 0.0511 - val_acc: 0.9829
Epoch 6/500
9142/9142 [==============================] - 5s 496us/step - loss: 0.0511 - acc: 0.9831 - val_loss: 0.0509 - val_acc: 0.9831
Epoch 7/500
9142/9142 [==============================] - 4s 465us/step - loss:

Epoch 60/500
9142/9142 [==============================] - 4s 472us/step - loss: 0.0365 - acc: 0.9867 - val_loss: 0.0433 - val_acc: 0.9857
Epoch 61/500
9142/9142 [==============================] - 5s 507us/step - loss: 0.0362 - acc: 0.9867 - val_loss: 0.0431 - val_acc: 0.9857
Epoch 62/500
9142/9142 [==============================] - 4s 486us/step - loss: 0.0362 - acc: 0.9868 - val_loss: 0.0429 - val_acc: 0.9857
Epoch 63/500
9142/9142 [==============================] - 4s 480us/step - loss: 0.0360 - acc: 0.9868 - val_loss: 0.0431 - val_acc: 0.9855
Epoch 64/500
9142/9142 [==============================] - 4s 450us/step - loss: 0.0361 - acc: 0.9867 - val_loss: 0.0434 - val_acc: 0.9855
Epoch 65/500
9142/9142 [==============================] - 4s 463us/step - loss: 0.0359 - acc: 0.9868 - val_loss: 0.0437 - val_acc: 0.9856
Epoch 66/500
9142/9142 [==============================] - 4s 461us/step - loss: 0.0360 - acc: 0.9868 - val_loss: 0.0432 - val_acc: 0.9857
Epoch 67/500
9142/9142 [==========

9142/9142 [==============================] - 5s 589us/step - loss: 0.0326 - acc: 0.9877 - val_loss: 0.0460 - val_acc: 0.9857
Epoch 120/500
9142/9142 [==============================] - 5s 494us/step - loss: 0.0328 - acc: 0.9876 - val_loss: 0.0458 - val_acc: 0.9857
Epoch 121/500
9142/9142 [==============================] - 4s 465us/step - loss: 0.0325 - acc: 0.9878 - val_loss: 0.0456 - val_acc: 0.9858
Epoch 122/500
9142/9142 [==============================] - 4s 447us/step - loss: 0.0323 - acc: 0.9879 - val_loss: 0.0459 - val_acc: 0.9858
Epoch 123/500
9142/9142 [==============================] - 5s 501us/step - loss: 0.0326 - acc: 0.9877 - val_loss: 0.0458 - val_acc: 0.9857
Epoch 124/500
9142/9142 [==============================] - 4s 446us/step - loss: 0.0323 - acc: 0.9878 - val_loss: 0.0460 - val_acc: 0.9855
Epoch 125/500
9142/9142 [==============================] - 4s 486us/step - loss: 0.0325 - acc: 0.9878 - val_loss: 0.0461 - val_acc: 0.9857
Epoch 126/500
9142/9142 [================

9142/9142 [==============================] - 4s 459us/step - loss: 0.0306 - acc: 0.9884 - val_loss: 0.0486 - val_acc: 0.9858
Epoch 179/500
9142/9142 [==============================] - 4s 448us/step - loss: 0.0307 - acc: 0.9883 - val_loss: 0.0490 - val_acc: 0.9858
Epoch 180/500
9142/9142 [==============================] - 4s 438us/step - loss: 0.0307 - acc: 0.9883 - val_loss: 0.0490 - val_acc: 0.9860
Epoch 181/500
9142/9142 [==============================] - 4s 445us/step - loss: 0.0305 - acc: 0.9884 - val_loss: 0.0492 - val_acc: 0.9858
Epoch 182/500
9142/9142 [==============================] - 4s 434us/step - loss: 0.0305 - acc: 0.9883 - val_loss: 0.0491 - val_acc: 0.9858
Epoch 183/500
9142/9142 [==============================] - 5s 520us/step - loss: 0.0303 - acc: 0.9884 - val_loss: 0.0490 - val_acc: 0.9858
Epoch 184/500
9142/9142 [==============================] - 5s 503us/step - loss: 0.0305 - acc: 0.9884 - val_loss: 0.0488 - val_acc: 0.9858
Epoch 185/500
9142/9142 [================

9142/9142 [==============================] - 4s 468us/step - loss: 0.0293 - acc: 0.9889 - val_loss: 0.0506 - val_acc: 0.9857
Epoch 238/500
9142/9142 [==============================] - 4s 459us/step - loss: 0.0292 - acc: 0.9887 - val_loss: 0.0519 - val_acc: 0.9857
Epoch 239/500
9142/9142 [==============================] - 4s 457us/step - loss: 0.0293 - acc: 0.9887 - val_loss: 0.0515 - val_acc: 0.9857
Epoch 240/500
9142/9142 [==============================] - 4s 457us/step - loss: 0.0291 - acc: 0.9889 - val_loss: 0.0518 - val_acc: 0.9857
Epoch 241/500
9142/9142 [==============================] - 4s 452us/step - loss: 0.0290 - acc: 0.9888 - val_loss: 0.0512 - val_acc: 0.9858
Epoch 242/500
9142/9142 [==============================] - 4s 448us/step - loss: 0.0291 - acc: 0.9888 - val_loss: 0.0516 - val_acc: 0.9857
Epoch 243/500
9142/9142 [==============================] - 4s 454us/step - loss: 0.0292 - acc: 0.9888 - val_loss: 0.0513 - val_acc: 0.9858
Epoch 244/500
9142/9142 [================

...

Epoch 208/500
9142/9142 [==============================] - 2s 231us/step - loss: 0.0455 - acc: 0.9840 - val_loss: 0.0442 - val_acc: 0.9842
Epoch 209/500
9142/9142 [==============================] - 2s 219us/step - loss: 0.0454 - acc: 0.9842 - val_loss: 0.0443 - val_acc: 0.9843
Epoch 210/500
9142/9142 [==============================] - 2s 220us/step - loss: 0.0455 - acc: 0.9841 - val_loss: 0.0443 - val_acc: 0.9844
Epoch 211/500
9142/9142 [==============================] - 2s 218us/step - loss: 0.0453 - acc: 0.9841 - val_loss: 0.0443 - val_acc: 0.9844
Epoch 212/500
9142/9142 [==============================] - 2s 217us/step - loss: 0.0454 - acc: 0.9842 - val_loss: 0.0442 - val_acc: 0.9844
Epoch 213/500
9142/9142 [==============================] - 2s 219us/step - loss: 0.0453 - acc: 0.9842 - val_loss: 0.0442 - val_acc: 0.9843
Epoch 214/500
9142/9142 [==============================] - 2s 219us/step - loss: 0.0452 - acc: 0.9841 - val_loss: 0.0442 - val_acc: 0.9843
Epoch 215/500
9142/9142 [==

Epoch 267/500
9142/9142 [==============================] - 3s 294us/step - loss: 0.0450 - acc: 0.9843 - val_loss: 0.0440 - val_acc: 0.9843
Epoch 268/500
9142/9142 [==============================] - 10s 1ms/step - loss: 0.0450 - acc: 0.9841 - val_loss: 0.0440 - val_acc: 0.9844
Epoch 269/500
9142/9142 [==============================] - 2s 218us/step - loss: 0.0452 - acc: 0.9842 - val_loss: 0.0441 - val_acc: 0.9843
Epoch 270/500
9142/9142 [==============================] - 2s 217us/step - loss: 0.0451 - acc: 0.9841 - val_loss: 0.0440 - val_acc: 0.9844
Epoch 271/500
9142/9142 [==============================] - 2s 214us/step - loss: 0.0451 - acc: 0.9842 - val_loss: 0.0440 - val_acc: 0.9844
Epoch 272/500
9142/9142 [==============================] - 2s 218us/step - loss: 0.0450 - acc: 0.9842 - val_loss: 0.0440 - val_acc: 0.9845
Epoch 273/500
9142/9142 [==============================] - 2s 218us/step - loss: 0.0451 - acc: 0.9842 - val_loss: 0.0440 - val_acc: 0.9843
Epoch 274/500
9142/9142 [===

Epoch 326/500
9142/9142 [==============================] - 2s 218us/step - loss: 0.0450 - acc: 0.9841 - val_loss: 0.0438 - val_acc: 0.9844
Epoch 327/500
9142/9142 [==============================] - 2s 216us/step - loss: 0.0450 - acc: 0.9842 - val_loss: 0.0437 - val_acc: 0.9845
Epoch 328/500
9142/9142 [==============================] - 2s 216us/step - loss: 0.0449 - acc: 0.9841 - val_loss: 0.0438 - val_acc: 0.9845
Epoch 329/500
9142/9142 [==============================] - 4s 407us/step - loss: 0.0450 - acc: 0.9841 - val_loss: 0.0438 - val_acc: 0.9843
Epoch 330/500
9142/9142 [==============================] - 5s 507us/step - loss: 0.0449 - acc: 0.9841 - val_loss: 0.0439 - val_acc: 0.9845
Epoch 331/500
9142/9142 [==============================] - 8s 888us/step - loss: 0.0451 - acc: 0.9842 - val_loss: 0.0437 - val_acc: 0.9844
Epoch 332/500
9142/9142 [==============================] - 2s 219us/step - loss: 0.0448 - acc: 0.9841 - val_loss: 0.0438 - val_acc: 0.9846
Epoch 333/500
9142/9142 [==

Epoch 385/500
9142/9142 [==============================] - 2s 218us/step - loss: 0.0447 - acc: 0.9842 - val_loss: 0.0436 - val_acc: 0.9846
Epoch 386/500
9142/9142 [==============================] - 2s 220us/step - loss: 0.0448 - acc: 0.9842 - val_loss: 0.0437 - val_acc: 0.9845
Epoch 387/500
9142/9142 [==============================] - 2s 218us/step - loss: 0.0446 - acc: 0.9843 - val_loss: 0.0437 - val_acc: 0.9844
Epoch 388/500
9142/9142 [==============================] - 2s 219us/step - loss: 0.0447 - acc: 0.9842 - val_loss: 0.0437 - val_acc: 0.9844
Epoch 389/500
9142/9142 [==============================] - 2s 220us/step - loss: 0.0448 - acc: 0.9841 - val_loss: 0.0436 - val_acc: 0.9845
Epoch 390/500
9142/9142 [==============================] - 2s 218us/step - loss: 0.0447 - acc: 0.9841 - val_loss: 0.0437 - val_acc: 0.9844
Epoch 391/500
9142/9142 [==============================] - 4s 413us/step - loss: 0.0448 - acc: 0.9842 - val_loss: 0.0437 - val_acc: 0.9844
Epoch 392/500
9142/9142 [==

Epoch 444/500
9142/9142 [==============================] - 2s 218us/step - loss: 0.0444 - acc: 0.9843 - val_loss: 0.0437 - val_acc: 0.9846
Epoch 445/500
9142/9142 [==============================] - 2s 219us/step - loss: 0.0445 - acc: 0.9843 - val_loss: 0.0436 - val_acc: 0.9846
Epoch 446/500
9142/9142 [==============================] - 2s 219us/step - loss: 0.0446 - acc: 0.9842 - val_loss: 0.0436 - val_acc: 0.9845
Epoch 447/500
9142/9142 [==============================] - 2s 217us/step - loss: 0.0445 - acc: 0.9842 - val_loss: 0.0435 - val_acc: 0.9846
Epoch 448/500
9142/9142 [==============================] - 2s 218us/step - loss: 0.0445 - acc: 0.9843 - val_loss: 0.0437 - val_acc: 0.9844
Epoch 449/500
9142/9142 [==============================] - 2s 262us/step - loss: 0.0444 - acc: 0.9842 - val_loss: 0.0436 - val_acc: 0.9845
Epoch 450/500
9142/9142 [==============================] - 3s 287us/step - loss: 0.0444 - acc: 0.9843 - val_loss: 0.0435 - val_acc: 0.9844
Epoch 451/500
9142/9142 [==